In [157]:
from ethnicolr import census_ln, pred_fl_reg_ln, pred_fl_reg_name
import pandas as pd
import numpy as np

In [233]:
def make_contributor_subset_chunk(df):
    """Subset the database to keep: 
    a. uniqueid (there are multiple IDs. Use one that gives a unique id for each contribution.) 
    b. columns related to name, and 
    c. year in which the contribution was made."""
    selected_cols = ['cycle', 'transaction.id',
     'date', 'contributor.name',
     'contributor.lname', 'contributor.fname',
     'contributor.mname', 'contributor.suffix',
     'contributor.title', 'contributor.ffname'
    ]
    subset_df = df
    new_col_names = [w.replace('.', '_') for w in selected_cols]
    new_cols_dict = dict(zip(selected_cols, new_col_names))
    subset_df.rename(columns=new_cols_dict, inplace=True)
    subset_df.rename(columns={'date':'contribution_year'}, inplace=True)
    return subset_df

In [249]:
def run_census_ln (subset_df, census_year):
    """Run the Census Ln Function."""
    has_last_name_df = subset_df[subset_df.contributor_lname.notnull()].copy()
    if len(has_last_name_df):
        return census_ln(has_last_name_df, 'contributor_lname', census_year)
    else:
        return pd.DataFrame()

In [250]:
def run_pred_fl_reg_ln(subset_df):
    """Run the pred_fl_reg_ln Function."""
    has_last_name_df = subset_df[subset_df.contributor_lname.notnull()].copy()
    if len(has_last_name_df):
        return pred_fl_reg_ln(has_last_name_df , 'contributor_lname')
    else:
        return pd.DataFrame()

In [251]:
def run_pred_fl_reg_name(subset_df):
    """Run Florida Data by Name"""
    has_last_name_df = subset_df[subset_df.contributor_lname.notnull()].copy()
    also_has_first_name_df = has_last_name_df[has_last_name_df.contributor_fname.notnull()].copy()
    if len(also_has_first_name_df):
        return pred_fl_reg_name(also_has_first_name_df, 'contributor_lname', 'contributor_fname')
    else:
        return pd.DataFrame()

In [307]:
import gzip
import os

def main_process_chunk(selected_csv):
    """Consolidate the steps in the data collation and sanitization."""
    m = re.match('.*?(\d+).*', selected_csv)
    if m:
        year = m.group(1)
    dime_database_file = selected_csv 
    selected_cols = ['cycle', 'transaction.id',
     'date', 'contributor.name',
     'contributor.lname', 'contributor.fname',
     'contributor.mname', 'contributor.suffix',
     'contributor.title', 'contributor.ffname'
    ]
    predictor_list = [{'name': 'census_ln_2000',
                              'function': lambda df: run_census_ln(df, 2000),
                              'outfile': None},
                            {'name': 'census_ln_2010',
                              'function': lambda df: run_census_ln(df, 2010),
                              'outfile': None},
                            {'name': 'pred_fl_reg_ln',
                              'function': run_pred_fl_reg_ln,
                              'outfile': None},
                            {'name': 'pred_fl_reg_name',
                              'function': run_pred_fl_reg_name,
                              'outfile': None},
                           ]
    dedup_keys = set()
    i = 0
    try:
        for data_df in pd.read_csv(dime_database_file, encoding='latin', usecols=selected_cols, low_memory=False, chunksize=100000):
            i += 1
            print(i)
            subset_df = make_contributor_subset_chunk(data_df)
            subset_df = subset_df[subset_df.contribution_year.notnull() & subset_df.contribution_year.str.contains('(?:19|20)\d\d\-\d\d\-\d\d.*')].copy()
            subset_df['annual_contrib_key'] = subset_df['contributor_name'] + pd.to_datetime(subset_df['contribution_year']).dt.year.astype(str)
            subset_df.drop_duplicates(subset=['annual_contrib_key'], keep=False, inplace=True)
            unique_contributors = subset_df.loc[~subset_df['annual_contrib_key'].isin(dedup_keys), :].copy()
            dedup_keys.update(subset_df['annual_contrib_key'].to_list())
            unique_contributors.drop(columns=['annual_contrib_key'], inplace=True)
            #print(len(dedup_keys))
            for p in predictor_list:
                results = p['function'](unique_contributors)
                if len(results) == 0:
                    continue
                if p['outfile'] is None:
                    fn = os.path.join('/opt/data/dime-race/', '{}_dime_{}_results.csv.gz'.format(year, p['name']))
                    print(fn)
                    p['outfile'] = gzip.open(fn, 'wt')
                    results.to_csv(p['outfile'], index=False, header=results.columns)
                else:
                    results.to_csv(p['outfile'], index=False, header=False, mode='at')
        for p in predictor_list:
            p['outfile'].close()
    except Exception as e:
        raise
        print('ERROR:', e)
    return subset_df

In [308]:
from os import path
from glob import glob
import re

sdf = None
for selected_csv in sorted(glob('/opt/data/dime-race/contribDB_*.csv.gz')):
    """Execute a single run of data fro dataset."""
    print(selected_csv)
    sdf = main_process_chunk(selected_csv)
    #break

/opt/data/dime-race/contribDB_2008.csv.gz
1
/opt/data/dime-race/2008_dime_census_ln_2000_results.csv.gz
/opt/data/dime-race/2008_dime_census_ln_2010_results.csv.gz
/opt/data/dime-race/2008_dime_pred_fl_reg_ln_results.csv.gz
/opt/data/dime-race/2008_dime_pred_fl_reg_name_results.csv.gz
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
/opt/data/dime-race/contribDB_2010.csv.gz
1
/opt/data/dime-race/2010

In [309]:
df = pd.read_csv('/opt/data/dime-race/2014_dime_pred_fl_reg_name_results.csv.gz')
df

,cycle,transaction_id,contribution_year,contributor_name,contributor_lname,contributor_fname,contributor_mname,contributor_suffix,contributor_title,contributor_ffname,race,asian,hispanic,nh_black,nh_white
0,2014,CA:1438763_INC13_2014_0_,2009-03-05,"hayes, janet gray",hayes,janet,gray,NaN,NaN,janet gray,nh_white,0.002450,0.005534,0.148020,0.843995
1,2014,CA:1438763_INC21_2014_0_,2009-03-20,"dailey, thomas m",dailey,thomas,m,NaN,NaN,thomas m,nh_white,0.002412,0.003960,0.038342,0.955286
2,2014,CA:1438763_INC24_2014_0_,2009-03-20,"giberson, margaret",giberson,margaret,NaN,NaN,NaN,margaret,nh_white,0.001794,0.005311,0.117157,0.875738
3,2014,CA:1438763_INC25_2014_0_,2009-03-20,"hooper, gregory f",hooper,gregory,f,NaN,NaN,gregory f,nh_white,0.002038,0.002623,0.149545,0.845794
4,2014,CA:1438763_INC26_2014_0_,2009-03-20,"lewis, william spencer",lewis,william,spencer,NaN,NaN,william spencer,nh_white,0.000905,0.004268,0.085702,0.909125
5,2014,CA:1438763_INC27_2014_0_,2009-03-20,"miller, kathleen mae",miller,kathleen,mae,NaN,NaN,kathleen mae,nh_white,0.002555,0.004558,0.033099,0.959788
6,2014,CA:1438763_INC29_2014_0_,2009-03-20,"walters, kim",walters,kim,NaN,NaN,NaN,kim,nh_white,0.007789,0.003099,0.160539,0.828573
7,2014,CA:1460289_INC100_2014_0_,2009-08-24,"boyarsky, jay s",boyarsky,jay,s,NaN,NaN,jay s,nh_white,0.001936,0.008696,0.003957,0.985411
8,2014,CA:1460289_INC105_2014_0_,2009-08-24,"francisco, ferdinand d",francisco,ferdinand,d,NaN,NaN,ferdinand d,nh_white,0.035581,0.242564,0.059238,0.662617
9,2014,CA:1460289_INC106_2014_0_,2009-08-24,"kim, tae yun",kim,tae,yun,NaN,NaN,tae yun,asian,0.993565,0.000234,0.001628,0.004573
